# Stock Analysis and AI-Powered Recommendation Tool

This notebook provides a comprehensive tool for analyzing stock performance and generating AI-powered insights. It performs the following functions:

1. Fetches historical stock data using Yahoo Finance API
2. Calculates key performance metrics (price, change, moving averages)
3. Uses Ollama (local LLM) to generate market analysis and recommendations
4. Saves the complete analysis to a text file for future reference

Default configuration is set for UKOG.L stock, but can be easily modified for any ticker.

## Setup and Dependencies

First, let's install the required packages:

In [25]:
# Install required packages
!pip install ollama yfinance pandas -q

In [26]:
# Import required libraries
import ollama                      # For AI-powered analysis
import yfinance as yf              # For fetching stock data
import pandas as pd                # For data manipulation
from typing import Optional, Dict, Any  # For type hints
from datetime import datetime, timedelta  # For date handling
import os                          # For file operations

## Global Configuration

These settings control the behavior of the analysis tool. You can modify these values to analyze different stocks or adjust the display format.

In [27]:
# Global configurations
OLLAMA_HOST = 'http://ollama:11434'  # Connection URL for Ollama API
TICKER = "VUSA.L"                    # Default stock ticker symbol
CURRENCY = "£"                       # Currency symbol for display formatting

## Data Collection and Analysis Functions

The following functions handle fetching stock data and calculating key metrics.

In [28]:
def get_stock_analysis(ticker: str = TICKER) -> Dict[str, Any]:
    """
    Retrieve and analyze stock data for the specified ticker.
    
    This function fetches one year of historical data and calculates key metrics
    including current price, day change percentage, 50-day moving average,
    and determines the current trend direction.
    
    Args:
        ticker (str): Stock ticker symbol (default: value from TICKER global variable)
        
    Returns:
        Dict[str, Any]: Dictionary containing calculated metrics and stock information
        None: If an error occurs during data retrieval or processing
    """
    try:
        # Get stock info
        stock = yf.Ticker(ticker)
        
        # Get historical data for the past year
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365)
        hist = stock.history(start=start_date, end=end_date)
        
        # Calculate key metrics
        current_price = hist['Close'][-1]  # Most recent closing price
        prev_close = hist['Close'][-2]     # Previous day's closing price
        day_change = ((current_price - prev_close) / prev_close) * 100  # Percentage change
        
        # Calculate 50-day moving average (technical indicator)
        ma50 = hist['Close'].rolling(window=50).mean().iloc[-1]
        
        # Determine trend direction based on price vs moving average
        short_term_trend = "UPWARD" if current_price > ma50 else "DOWNWARD"
        
        # Return comprehensive data dictionary
        return {
            "ticker": ticker,
            "current_price": round(current_price, 2),
            "previous_close": round(prev_close, 2),
            "day_change": round(day_change, 2),
            "ma50": round(ma50, 2),
            "trend": short_term_trend,
            "volume": int(hist['Volume'][-1]),
            "last_updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    
    except Exception as e:
        print(f"Error retrieving stock data: {str(e)}")
        return None

In [29]:
def get_stock_info(ticker: str = TICKER) -> str:
    """
    Get stock name and description for better prompts.
    
    This function retrieves the company's full name to create more
    informative prompts for the AI model.
    
    Args:
        ticker (str): Stock ticker symbol
        
    Returns:
        str: Company name and ticker in format "Company Name (TICKER)"
             or just the ticker if retrieval fails
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return f"{info.get('longName', ticker)} ({ticker})"
    except:
        return ticker

## AI Analysis Functions

These functions handle the interaction with the Ollama LLM to generate insights.

In [30]:
def generate_response(prompt: str, model: str = "llama3.1:8b-instruct-q8_0") -> Optional[str]:
    """
    Generate a response using the Ollama model.
    
    This function sends a prompt to the specified Ollama model and returns
    the generated text response.
    
    Args:
        prompt (str): The text prompt to send to the model
        model (str): The Ollama model identifier to use (default: llama3.1:8b-instruct-q8_0)
        
    Returns:
        Optional[str]: The generated text response or None if an error occurs
    """
    try:
        # Initialize Ollama client with the configured host
        client = ollama.Client(host=OLLAMA_HOST)
        
        # Generate response from the model
        response = client.generate(model=model, prompt=prompt)
        
        return response['response']
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

## Output and File Handling

In [31]:
def save_to_file(data: Dict[str, Any], analysis: str, recommendation: str) -> str:
    """
    Save the analysis to a formatted text file.
    
    This function creates a well-structured report file containing all the
    stock data, AI-generated analysis, and investment recommendation.
    
    Args:
        data (Dict[str, Any]): Stock data dictionary from get_stock_analysis
        analysis (str): AI-generated market analysis text
        recommendation (str): AI-generated investment recommendation
        
    Returns:
        str: Path to the saved file
    """
    # Create timestamp and filename with format: TICKER_analysis_YYYYMMDD_HHMMSS.txt
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"outputs/{data['ticker']}_analysis_{timestamp}.txt"
    
    # Create outputs directory if it doesn't exist
    os.makedirs('outputs', exist_ok=True)
    
    # Write formatted report to file
    with open(filename, 'w', encoding='utf-8') as f:
        # Header section
        f.write(f"{data['ticker']} Analysis Report\n")
        f.write(f"Generated: {data['last_updated']}\n")
        f.write("=" * 50 + "\n\n")
        
        # Market data section
        f.write("Market Data:\n")
        f.write(f"Current Price: {CURRENCY}{data['current_price']}\n")
        f.write(f"Previous Close: {CURRENCY}{data['previous_close']}\n")
        f.write(f"Day Change: {data['day_change']}%\n")
        f.write(f"50-day MA: {CURRENCY}{data['ma50']}\n")
        f.write(f"Current Trend: {data['trend']}\n")
        f.write(f"Volume: {data['volume']:,}\n")
        f.write("=" * 50 + "\n\n")
        
        # Analysis section
        f.write("Analysis:\n")
        f.write(f"{analysis}\n")
        f.write("=" * 50 + "\n\n")
        
        # Recommendation section
        f.write("Purchase Recommendation:\n")
        f.write(f"{recommendation}\n")
        f.write("=" * 50 + "\n")
    
    return filename

## Main Execution Function

This function orchestrates the entire analysis process.

In [32]:
def main(ticker: str = TICKER):
    """
    Main function that orchestrates the stock analysis workflow.
    
    This function coordinates the entire process:
    1. Fetches stock data
    2. Generates AI prompts based on the data
    3. Retrieves AI-generated analysis and recommendations
    4. Displays results and saves to file
    
    Args:
        ticker (str): Stock ticker symbol to analyze (default: global TICKER value)
    """
    print(f"Analyzing {ticker}...")
    
    # Get stock data and company name
    stock_data = get_stock_analysis(ticker)
    stock_name = get_stock_info(ticker)
    
    if stock_data:
        # Generate analysis prompt with key metrics
        analysis_prompt = f"""
        Analyze this {stock_name} data:
        - Current Price: {CURRENCY}{stock_data['current_price']}
        - Day Change: {stock_data['day_change']}%
        - 50-day Moving Average: {CURRENCY}{stock_data['ma50']}
        - Trend: {stock_data['trend']}
        
        Provide a brief market analysis focusing on current performance.
        """
        
        # Generate recommendation prompt
        recommendation_prompt = f"""
        Based on this {stock_name} data:
        - Current Price: {CURRENCY}{stock_data['current_price']}
        - 50-day MA: {CURRENCY}{stock_data['ma50']}
        - Trend: {stock_data['trend']}
        
        Provide a clear BUY, HOLD, or WAIT recommendation with a brief explanation.
        Consider if today is a good day to purchase additional shares.
        """
        
        # Get AI-generated responses
        analysis = generate_response(analysis_prompt)
        recommendation = generate_response(recommendation_prompt)
        
        if analysis and recommendation:
            # Print results to notebook output
            print(f"\nCurrent Market Data for {stock_name}:")
            print(f"Price: {CURRENCY}{stock_data['current_price']}")
            print(f"Day Change: {stock_data['day_change']}%")
            print(f"Trend: {stock_data['trend']}")
            print("\nAnalysis:")
            print(analysis)
            print("\nRecommendation:")
            print(recommendation)
            
            # Save complete report to file
            saved_file = save_to_file(stock_data, analysis, recommendation)
            print(f"\nFull report saved to: {saved_file}")

## Run the Analysis

Execute the main function to perform the stock analysis. You can modify the ticker symbol to analyze different stocks.

In [33]:
if __name__ == "__main__":
    # You can easily change the ticker here or pass it as an argument
    # Example usage:
    # main("AAPL")    # for Apple
    # main("GOOGL")   # for Google
    # main("VUSA.L")  # for Vanguard S&P 500 UCITS ETF
    main(TICKER)

Analyzing VUSA.L...


/tmp/ipykernel_23520/1658910072.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = hist['Close'][-1]  # Most recent closing price
/tmp/ipykernel_23520/1658910072.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prev_close = hist['Close'][-2]     # Previous day's closing price
/tmp/ipykernel_23520/1658910072.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "volume": int(hist['Volume'][-1]),



Current Market Data for Vanguard S&P 500 UCITS ETF (VUSA.L):
Price: £83.68
Day Change: -1.99%
Trend: DOWNWARD

Analysis:
**Market Analysis: Vanguard S&P 500 UCITS ETF (VUSA.L)**

The Vanguard S&P 500 UCITS ETF (VUSA.L) has experienced a significant decline in its share price over the past day, with a drop of -1.99%. This downward trend is also reflected in its moving average, as the 50-day MA (£90.89) currently sits above the current price (£83.68), indicating a sell-off.

**Key Observations:**

1. **Downward Trend**: The overall trend for VUSA.L is DOWNWARD, which suggests that investors are becoming increasingly bearish on the market.
2. **Price Gap**: The current price has dropped below the 50-day MA, creating a price gap of approximately £7.21 (£90.89 - £83.68). This gap indicates a potential area of support for buyers to take advantage of.
3. **Volatility**: The day change of -1.99% suggests that the market is experiencing increased volatility, which may be attributed to various 